In [1]:
import torch

In [2]:
from transformers import AutoModel, AutoTokenizer

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [5]:
from typing import List
def tokenize(queries: List[str], device: torch.device):
    formatted_queries = [f'query: {sentence}' for sentence in queries]
    tokenized_sentences = tokenizer(
        formatted_queries, max_length=512,
        padding=True, truncation=True,
        return_tensors='pt'
    )
    tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(device)
    tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(device)
    return tokenized_sentences

In [6]:
from torch import Tensor

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
from tqdm.auto import tqdm

model.eval()
outputs = []

with torch.no_grad():
    for i in tqdm(range(len(formatted_queries))):
        batch = {'input_ids': tokenized_sentences['input_ids'][i].unsqueeze(0), 'attention_mask': tokenized_sentences['attention_mask'][i].unsqueeze(0)}
        out = model(**batch)
        embeddings = average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'][i].unsqueeze(0))
        outputs.append(embeddings.squeeze(0))

In [7]:
del model
del tokenizer

torch.cuda.empty_cache()

In [8]:
import gc
gc.collect()

85